In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
DATA_PATH = "/content/drive/MyDrive/final_pj/final_project_data/"
DATA_PATH

'/content/drive/MyDrive/final_pj/final_project_data/'

In [3]:
SEED = 42

In [28]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [29]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.0_1106.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.0_1106.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 1504), (12225, 1504))

In [30]:
train_ft.isnull().sum().sum()

0

In [31]:
test_ft.isnull().sum().sum()

0

In [32]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1503), (12225, 1503))

In [33]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_대분류,22
주구매_중분류,246


In [34]:
cols

['주구매지점', '주구매_대분류', '주구매_중분류']

In [35]:
%pip install category_encoders

In [36]:
import category_encoders as ce
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[cols])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[cols])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1775), (12225, 1775))

In [37]:
cols

['주구매지점', '주구매_대분류', '주구매_중분류']

In [38]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1772), (12225, 1772))

In [39]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [40]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

###StandardScaler

In [17]:
#scaler = StandardScaler()
#scaler.fit(train_ft)
#train_ft[train_ft.columns] = scaler.transform(train_ft)
#test_ft[test_ft.columns] = scaler.transform(test_ft)

In [41]:
scaler = MinMaxScaler()
scaler.fit(train_ft)
train_ft[train_ft.columns] = scaler.transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [42]:
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,...,주구매_중분류_237,주구매_중분류_238,주구매_중분류_239,주구매_중분류_240,주구매_중분류_241,주구매_중분류_242,주구매_중분류_243,주구매_중분류_244,주구매_중분류_245,주구매_중분류_246
0,0.041494,0.130682,0.250000,0.750000,0.032258,0.029014,0.050000,0.250000,0.400000,0.300000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.082988,0.090909,0.023810,0.976190,0.006452,0.079304,0.357143,0.166667,0.357143,0.119048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.228216,0.034091,0.210526,0.789474,0.154839,0.174081,0.464912,0.140351,0.175439,0.219298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.377593,0.017045,0.189573,0.810427,0.258065,0.330754,0.379147,0.180095,0.236967,0.203791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.107884,0.062500,0.258065,0.741935,0.103226,0.088975,0.112903,0.612903,0.209677,0.064516,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


In [27]:
#standard_scaler
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

# 파라미터 설정
params = {
    "random_state": 42,
    "learning_rate": 0.1,  # 학습률
    "n_estimators": 500,   # 트리 개수
    "max_depth": 5,        # 최대 깊이
    "min_child_weight": 3, # 최소 자식 가중치
    "subsample": 0.8,      # 샘플링 비율
    "colsample_bytree": 0.8, # 특성 샘플링 비율
    "gamma": 0.1,           # 최소 손실 감소
    "n_jobs": -1,
    "early_stopping_rounds": 50,
    "eval_metric": "logloss"
}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7155987156775787


In [46]:
#min-max scaler
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

# 파라미터 설정
params = {
    "random_state": 42,
    "learning_rate": 0.1,  # 학습률
    "n_estimators": 500,   # 트리 개수
    "max_depth": 5,        # 최대 깊이
    "min_child_weight": 3, # 최소 자식 가중치
    "subsample": 0.8,      # 샘플링 비율
    "colsample_bytree": 0.8, # 특성 샘플링 비율
    "gamma": 0.1,           # 최소 손실 감소
    "n_jobs": -1,
    "early_stopping_rounds": 50,
    "eval_metric": "logloss"
}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7155987156775787


categorical_feature 추가
범주형 피처 최적화



In [ ]:
feature_importances = np.mean(feature_importances, axis=0)  # 각 모델의 피처 중요도를 평균
feature_importance_df = pd.DataFrame({
    'Feature': train_ft.columns,  # 피처 이름
    'Importance': feature_importances  # 피처 중요도
})

# 중요도가 높은 순서대로 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
feature_importance_df.head(20)

In [47]:
model = XGBClassifier(**params)
model.fit(train_ft, target)

ValueError: Must have at least 1 validation dataset for early stopping.

In [ ]:
pred = model.predict(test_ft)  # test_data는 테스트 데이터셋입니다.
pred

In [ ]:
submit["target"] = pred
submit

In [ ]:
submit.to_csv(f"{DATA_PATH}xgboost_1106_submit.csv",index=False)